In [ ]:
!pip install z3-solver
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 78 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.1 MB/s 


In [ ]:
from google.colab import drive
from google.colab import files
from z3 import *
import matplotlib.pyplot as plt
import numpy as np
import xlsxwriter

INSTANCE_PATH = '/content/gdrive/MyDrive/unibo/CombinatorialProject/VLSI_instances/'
RESULT_PATH = "/content/gdrive/MyDrive/unibo/CombinatorialProject/out/"

drive.mount('/content/gdrive', force_remount=True)



Mounted at /content/gdrive


## **Utility functions**

In [ ]:

def read_instance(n):
    """ Read one instance file and return a dict containing the values
    Args:
      n(int): number of the instance
    Returns:
      dict: containing the instance values
    """
    with open(INSTANCE_PATH + 'ins-' + str(n) + '.txt') as f:
        contents = f.read()
    f.close()

    contents = contents.splitlines()
    w = int(contents[0])
    n = int(contents[1])
    mtr = []
    for i in range(2, n + 2):
        mtr.append([int(x) for x in contents[i].split()])

    return {'w': w, 'n': n, 'mtr': mtr}



def write_results(num_instance, instance, h,corners_x, corners_y, rotation = False, rotated = []):
    """ Write the results of the computed instance to a txt file
    Args:
      num_instance(int): number of the instance
      instance(dict): dictionary containing the values of the instance
      h(int): plate's height
      corners_x(list:int): list containing the x coordinate of the bottom-left corner for each block
      corners_y(list:int): list containing the y coordinate of the bottom-left corner for each block
      rotation(bool): true if the results were computed with rotation enabled, false otherwise
      rotated(list:bool): list containing for each block if it has been rotated, 
                          the list must be empty if rotation is false
                          
    """

    path =''
    if rotation:
      path = RESULT_PATH + '/solutionRotation/out-rot-' + str(num_instance) + '.txt'
    else:
      path = RESULT_PATH + '/solutionNoRotation/out-' + str(num_instance) + '.txt'

    n = instance['n']
    w = instance['w']
    widths = instance['mtr'][:,0]
    heights = instance['mtr'][:,1]

    with open(path, 'w') as f:
        f.write(str(w) + " " + str(h) + "\n")
        f.write(str(n) + '\n')
        for i in range(n):
            if rotation and rotated[i]:
               f.write(str(heights[i]) + " " + str(widths[i]) + " "
                    +  str(corners_x[i]) + " " + str(corners_y[i]) + "\n")
            else:
              f.write(str(widths[i]) + " " + str(heights[i]) + " "
                      +  str(corners_x[i]) + " " + str(corners_y[i]) + "\n")
    f.close()



def show_results(num_instance, rotation=False, rotated=None, only_write_img = False):
    """ Function that read the txt file containing the solution for a single instance.
        Then, plot and save as image the result.
    Args:
      num_instance(int): number of the instance
      rotation(bool): true if the results were computed with rotation enabled, false otherwise
      rotated(list:bool): list containing for each block if it has been rotated, 
                          the list must be empty if rotation is false
                          
    """
    path = ''
    if rotation:
        path = RESULT_PATH  + 'solutionRotation/out-rot-' + str(num_instance) + '.txt'
    else:
        path = RESULT_PATH  + 'solutionNoRotation/out-' + str(num_instance) + '.txt'

    with open(path) as f:
        contents = f.read()
    f.close()

    contents = contents.splitlines()
    w, h = contents[0].split()
    n = int(contents[1])
    w = int(w)
    h = int(h)

    widths = []
    heights = []
    corners_x = []
    corners_y = []

    for k in range(2, n + 2):
        width, height, x, y = contents[k].split()
        widths.append(int(width))
        heights.append(int(height))
        corners_x.append(int(x))
        corners_y.append(int(y))

    title = 'Solution instance ' + str(num_instance)
    if rotation:
        title += ' with rotation'

    fig, ax = plt.subplots()
    fig.set_size_inches(420/25.4, 297/25.4)

    ax.set_title(title)

    for i in range(n):
        color = np.array(np.random.choice(range(30, 231), size=3))
        color = color / 255
        rect = plt.Rectangle((corners_x[i], corners_y[i]), widths[i], heights[i],
                             facecolor=color, edgecolor='black', linestyle='solid', linewidth=2.5)
        ax.add_patch(rect)

        if rotation and rotated[i]:
            circle = plt.Circle((corners_x[i] + widths[i] / 2, corners_y[i] + heights[i] / 2), 0.20, color='w',zorder=10)
            ax.add_patch(circle)

    ax.set(xlim=(0, w), ylim=(0, h))
    ax.set_aspect('equal')
    ax.grid(color='black', linewidth=0.75, linestyle='--')

    plt.xticks(np.arange(0, w + 1, 1), rotation=90)
    plt.yticks(np.arange(0, h + 1, 1))

    path = ''
    if rotation:
        path = RESULT_PATH + 'solutionRotation/images/out-rot-' + str(num_instance) + '.png'
    else:
        path = RESULT_PATH + 'solutionNoRotation/images/out-' + str(num_instance) + '.png'
    plt.savefig(path)
    if not only_write_img:
        plt.show()

    plt.close()


def order_by_area(shape_matrix):
    """ function that sorts the matrix from largest to smallest using the area of each block
    Args:
      shape_matrix(matrix:int): matrix containing the shape of each block
    Returns:
      matrix: the sorted matrix
    """
    n = len(shape_matrix[:, 0])
    for i in range(n - 1):
        for j in range(i + 1, n):
            if (shape_matrix[i, 0] * shape_matrix[i, 1]) < (shape_matrix[j, 0] * shape_matrix[j, 1]):
                tmp = np.copy(shape_matrix[i])
                shape_matrix[i] = np.copy(shape_matrix[j])
                shape_matrix[j] = np.copy(tmp)

    return shape_matrix


def get_upper_bound(instance):
  """ Compute the upper bound for the plate
    Args:
      instance(dict): dictionary containing the values of the instance
    Returns:
      int: the computed upper bound for the plate
  """

  w = instance['w']
  n = instance['n']
  shape_matrix = instance['mtr']

  combination_indexes = []
  combination_indexes.append(0)

  width_sum = shape_matrix[0][0]
  width_sum_no_last = 0

  for i in range(1, n):
    width_sum_no_last = width_sum
    width_sum += shape_matrix[i][0]
   
    if( ( (width_sum - 1 ) % w) < ( (width_sum_no_last - 1 )% w) ) :
      combination_indexes.append(i)

  combination_indexes.append(n-1)

  max_h = 0
  for i in range (len(combination_indexes) -1):
    if combination_indexes[i] != combination_indexes[i+1]:
      max_h += np.max(shape_matrix[combination_indexes[i]:(combination_indexes[i+1]),1])
    else:
      max_h += shape_matrix[combination_indexes[i],1]
    

  return int(max_h)

def get_lower_bound(instance, rotation = False):
    """ Compute the lower bound for the plate
    Args:
      instance(dict): dictionary containing the values of the instance
      rotation(bool): true if rotation enabled, false otherwise
    Returns:
      int: the computed lower bound for the plate
    """
    w = instance['w']
    n = instance['n']
    shape_matrix = instance['mtr']
    min_h = 0
    for k in range(n):
        min_h += shape_matrix[k][0] * shape_matrix[k][1]
    min_h = np.ceil(min_h / w)

    if rotation:
      min_between_h_w = np.min([np.max(shape_matrix[:, 1]), np.max(shape_matrix[:, 0])])
      return int(np.max([min_h, min_between_h_w]))
    else:
      return int(np.max([min_h, np.max(shape_matrix[:, 1])]))



def prepare_instance(instance_num):
    """ Read one instance file, then sort the block by area and return a dictionary 
        containing the instance's values
    Args:
      n(int): number of the instance
    Returns:
      dict: containing the instance values
    """
    
    instance = read_instance(instance_num)
    instance['mtr'] = order_by_area(np.asarray(instance['mtr']))
    return instance

## **Encoding**

In [ ]:
def smt_encode(instance, h, rotation = False, symmtry_breaking = False ):
    """ Function that encode the VLSI design problem of the specified instance
    Args:
      instance(dict): dictionary containing the values of the instance
      rotation(bool): true to enable blocks rotation, false otherwise
      symmtry_breaking(bool): true to use symmetry breaking constraints, false otherwise
    Returns:
      z3.Goal: goal object containing the smt encoding of the problem
    """
    w = instance['w']
    n = instance['n']

    widths_arr = instance['mtr'][:,0]
    heights_arr = instance['mtr'][:,1]

    # create Z3 variables to represent the VLSI design problem
    w_val = IntVal(w)
    h_val = IntVal(h)
    corner_x = [Int(f"x_{i}") for i in range(n)]
    corner_y = [Int(f"y_{i}") for i in range(n)]
    widths = [Int(f"w_{i}") for i in range(n)]
    heights = [Int(f"h_{i}") for i in range(n)]

    g = Goal()

    if rotation:
      print('- rotation enabled -')
      #create z3.Bool variables to allow block rotation
      rotated = [Bool(f'r_{i}') for i in range(n)]

      g.add([Or(And(Not(rotated[i]) , widths[i] == int(widths_arr[i]), heights[i] == int(heights_arr[i])),
                And(rotated[i], widths[i] == int(heights_arr[i]), heights[i] == int(widths_arr[i]))) for i in range(n)])
      
      
      # auxiliary constraints for rotation
      '''
      # (original_width = original_height) ==> rotated == False
      g.add([Not(And(int(widths_arr[i]) == int(heights_arr[i]), rotated[i])) for i in range(n)])\

      # original_height > plate_width ==> rotated == False
      g.add([Not(And(int(heights_arr[i]) > w , rotated[i])) for i in range(n)])

      # original_width > plate_heght ==> rotated == False
      g.add([Not(And(int(widths_arr[i]) > h , rotated[i])) for i in range(n)])
      '''
    else:
      g.add([And(widths[i] == int(widths_arr[i]), heights[i] == int(heights_arr[i])) for i in range(n)])


    # add boundary constraint for each block
    g.add([And(corner_x[i] >= 0, corner_y[i] >= 0, 
              corner_x[i] <=  w_val  - widths[i] , corner_y[i]  <= h_val - heights[i]) 
            for i in range(n)])
    
    
    # no overlap constraint for each pair of blocks with NotAnd formulation
    g.add([Not(And( corner_x[i] - corner_x[j] <= widths[j] - 1,
                    corner_x[j] - corner_x[i] <= widths[i] - 1,
                    corner_y[i] - corner_y[j] <= heights[j] - 1,
                    corner_y[j] - corner_y[i] <= heights[i] - 1 ))
            for i in range(n) for j in range(i+1, n)])
    
    '''
    # no overlap constraint for each pair of blocks with Or formulation
    g.add([Or( corner_x[i] - corner_x[j] <= - widths[i],
                corner_y[i] - corner_y[j] <= - heights[i],
                corner_x[j] - corner_x[i] <= - widths[j],
                corner_y[j] - corner_y[i] <= - heights[j])
            for i in range(n) for j in range(i+1, n)])
    ''' 
    
    if symmtry_breaking:
        print('- Adding symmetry breaking constraints')
        g.add(use_symmetry_breaking(n,w, h, widths, heights, corner_x, corner_y))
    
    
    return g


In [ ]:
def use_symmetry_breaking(n,w, h, widths, heights, corner_x, corner_y):
    """ Return the encoding of the symmetry breaking constraint
    Args:
      n(int): blocks num
      w(int): plate's width
      h(int): plate's height
      widths(list:z3.Int): list of z3 variables representing block widths
      heights(z3.int): list of z3 variables representing block heights
      corner_x(list:int): list containing the x coordinate of the bottom-left corner for each block
      corner_y(list:int): list containing the y coordinate of the bottom-left corner for each block

    returns:
      z3.And: encoding of the symmetry breaking constraint

    """
    # fix the first block to the bottom-left of the second block
    #ordering_constraint = And( corner_x[0] <= corner_x[1], corner_y[0] <= corner_y[1])

    # Fix the first block in the bottom-left part of the plate
    width_bound = int(np.floor(w/2))
    height_bound = int(np.floor(h/2))
    reduce_domain_first_block = And( corner_x[0] <= width_bound, corner_y[0] <= height_bound)

    '''
    # Ordering constraint between block with same widths 
    ordering_widths = [Implies(And(corner_x[i] == corner_x[j], widths[i] == widths[j]),
                            corner_y[i] <= corner_y[j])
                    for i in range(n) for j in range(i+1, n)]
    
    # Ordering constraint between block with same heights 
    ordering_heights = [Implies(And(corner_y[i] == corner_y[j], heights[i] == heights[j]),
                            corner_x[i] <= corner_x[j])
                    for i in range(n) for j in range(i+1,n)]

    used_constraint = And(reduce_domain_first_block, *ordering_widths, *ordering_heights)
    '''
    return reduce_domain_first_block

In [ ]:
def smt_check(smt_encoding, time_limit=None):
    """ Function that checks if the encoding passed as input is satisfaible
    Args:
      smt_encoding(z3.Goal): encoding of VLSI design problem for an instance
      time_limit(int): time limit in second for the solver, None if the time limit is not needed
    Returns:
      bool: True if the encoding is satisfaible, false otherwise
      z3.model: the model if the encoding is satisfaible, None otherwise
      z3.statistics: the statistics collected by the solver
    """

    # preprocessing tactics in order to simplify the encoding passed as input
    t = Then(
        Repeat(With(Tactic('simplify'), cache_all = True)),
        Repeat('solve-eqs'),
        Repeat('propagate-ineqs'),
        Repeat('propagate-values'),
        'symmetry-reduce',
        'purify-arith'
    )

    # the theory solver used to solve the goal
    #theory_solver = Tactic('qflia')
    theory_solver = Tactic('qfidl')
    #theory_solver = Tactic('default')

    final_tactic = Then(t, theory_solver)
    s = final_tactic.solver()
    
    if time_limit is not None:
        print('Solver timout: ', time_limit, 's')
        s.set("timeout", int(time_limit * 1000))

    s.add(smt_encoding)

    if s.check() == sat:
        return True, s.model(), s.statistics()
    else:
        return False, None, s.statistics()


In [ ]:
def get_solution(n, model, rotation = False):
  """ Function that decode the results from the model
    Args:
      n(int): blocks num
      model(z3.model): model that represents the solution found
      rotation(bool): true if the results were computed with rotation enabled, false otherwise
      
    Returns:
      corners_x(list:int): list containing the x coordinate of the bottom-left corner for each block
      corners_y(list:int): list containing the y coordinate of the bottom-left corner for each block
      
      rotated(list:bool): list containing for each block if it has been rotated, 
                          the list is empty if rotation is false
  """

  corner_x_vars = [Int(f"x_{i}") for i in range(n)]
  corner_y_vars = [Int(f"y_{i}") for i in range(n)]
  widths_vars = [Int(f"w_{i}") for i in range(n)]
  heights_vars = [Int(f"h_{i}") for i in range(n)]
  rotated_vars = [Bool(f'r_{i}') for i in range(n)]
  
  corner_x = []
  corner_y = []
  widths = []
  heights = []
  rotated = []

  for i in range(n):
    corner_x.append(model.evaluate(corner_x_vars[i]).as_long())
    corner_y.append(model.evaluate(corner_y_vars[i]).as_long())
    widths.append(model.evaluate(widths_vars[i]).as_long())
    heights.append(model.evaluate(heights_vars[i]).as_long())
    
    if rotation:
      if model.evaluate(rotated_vars[i]) == True:
        rotated.append(True)
      else:
        rotated.append(False)
  return corner_x, corner_y, rotated


## **Experiment**

### **Experiment.Matherials**

In [ ]:
def solve_instances(num_instances, time_limit = None, rotation = False, symmetry_breaking = False, verbose = False, count = 0):
  """ Function that allows to solve the specified instances with different configurations for the encoding and the solver.
        Furthermore, this function collects and saves all data in xlsx format
    Args:
      num_instances(list:int): list containing the number of instance to solve
      time_limit(int): time limit in second for the solver, None if the time limit is not needed
      rotation(bool): true to enable blocks rotation, false otherwise
      symmetry_breaking(bool): true to use symmetry breaking constraints, false otherwise
      verbose(bool): true to print the statistics collected by the solver, false otherwise
      count(int): num of the experiment
                          
  """
  
  # list used to store all the data and statistics collected for every instance
  all_num_instances = []
  all_solutions = []
  all_is_optimal_solution = []
  all_statistics = []


  for i in num_instances:
    print('\n***Instance num: ', str(i), '***\n')

    all_num_instances.append(i)

    instance = prepare_instance(i)
    min_h = get_lower_bound(instance, rotation)
    max_h = get_upper_bound(instance)
    
    print('Plate width -> ', instance['w'])
    print('Min h -> ', min_h)
    print('Max h -> ', max_h)

    count_attempt = 1;
    test_h = (min_h + max_h)//2
    model = None
    statistics = None
    time = 0
    last_h = 0

    while (min_h <= test_h and test_h <= max_h):
      print('\n - Attempt num -> ', count_attempt)
      print(' - Tested h -> ', test_h)
      encoding = smt_encode(instance, test_h, rotation, symmetry_breaking)

      if time_limit != None:
        is_sat, model_tmp, statistics_tmp = smt_check(encoding, time_limit - time)
      else: 
        is_sat, model_tmp, statistics_tmp = smt_check(encoding, None)


      time_tmp = statistics_tmp.get_key_value('time')
      time += time_tmp

      print('Expired time for this attempt -> ', time_tmp,'s')
      if is_sat:
        print('**SAT**')
        max_h = test_h - 1
        last_h = test_h
        test_h = (min_h + max_h)//2
        model = model_tmp
        statistics = statistics_tmp
      else:
        print('**UNSAT**')
        min_h = test_h + 1 
        test_h = (min_h + max_h)//2
      
      count_attempt += 1

      if time_limit != None and time >= time_limit:
        break
        
    if (time_limit == None or time <= time_limit) and model != None:
      print('Optimal solution founded in ', round(time, 3), 's, h -> ', test_h)

      all_is_optimal_solution.append(True)
      all_solutions.append(last_h)

      if verbose and statistics is not None:
          print('Statistics from z3 solver')
          print(statistics)
      
      all_statistics.append(statistics)

      corner_x, corner_y, rotated = get_solution(instance['n'], model, rotation)
      
      write_results(i, instance,last_h, corner_x, corner_y, rotation, rotated)
      show_results(i, rotation, rotated)
        
    else:
      print('No optimal solution in' + str(time_limit) + 's founded')
      all_is_optimal_solution.append(False)
      

      if model != None:
        print('Last suboptimal solution')

        if verbose and statistics is not None:
          print('Statistics from z3 solver')
          print(statistics)
        
        all_statistics.append(statistics)
        all_solutions.append(last_h)
        
        corner_x, corner_y, rotated = get_solution(instance['n'], model, rotation)
        write_results(i, instance,last_h, corner_x, corner_y, rotation, rotated)
        show_results(i, rotation, rotated)
      else:
        all_statistics.append(None)
        all_solutions.append(0)
  

  # write the excel file containing all the information
  compute_statistics(rotation, symmetry_breaking ,all_num_instances, all_solutions, all_is_optimal_solution, all_statistics, count)
    



In [ ]:
def compute_statistics(rotation, symmetry_breaking ,all_num_instances, all_solutions, all_is_optimal_solution, all_statistics, count):
  """ Function that allows to solve the specified instances with different configurations for the encoding and the solver.
        Furthermore, this function collects and saves all data in xlsx format
    Args:
      rotation(bool): true if blocks rotation was enabled, false otherwise
      symmtry_breaking(bool): true if symmetry breaking constraints were enabled, false otherwise
      all_num_instances(list:int): list containing the num of the solved instances
      all_solutions(list:int): List containing the last solution found
      all_is_optimal_solution(list:bool): list containing boolean to represent that the solution 
                                          found was the optimal one for each instance
      all_statistics(list;z3.statistics): list containing the collected statistics by the solver for each instance                                    
      count(int): num of the experiment
                          
  """
  title = '/content/gdrive/MyDrive/unibo/CombinatorialProject/collected_data/' + str(count) +'-IDL_NotAnd'

  if rotation:
    title += '_With_rotation'
  else:
    title += '_no_rotation'
  
  if symmetry_breaking:
    title += '_with_symmetry_breaking'
  else:
    title += '_no_symmetry_breaking'

  title += '.xlsx'

  print(title)

  workbook = xlsxwriter.Workbook(title)
  worksheet = workbook.add_worksheet()

  # Add a bold format to use to highlight cells.
  bold = workbook.add_format({'bold': True})
  cell_format_red_bg = workbook.add_format({'bg_color': 'red'})
 
  # Write some data headers.
  worksheet.write('A1', 'Instance', bold)
  worksheet.write('B1', 'Time', bold)
  worksheet.write('C1', 'Conflicts', bold)
  worksheet.write('D1', 'Propagations', bold)
  worksheet.write('E1', 'Best solution', bold)

  start_row =  1
  for i in range(len(all_num_instances)):
    worksheet.write(start_row +i, 0, all_num_instances[i])

    if all_is_optimal_solution[i]:
      time = round(all_statistics[i].get_key_value('time'), 3)
      worksheet.write(start_row +i, 1, time)
      worksheet.write(start_row +i, 4, all_solutions[i])
    else:
      worksheet.write(start_row +i, 1, 'Timeout', cell_format_red_bg)
      worksheet.write(start_row +i, 4, all_solutions[i], cell_format_red_bg)


    if all_statistics[i] is not None and 'conflicts' in all_statistics[i].keys():
      conflicts = int(all_statistics[i].get_key_value('conflicts'))
      worksheet.write(start_row +i, 2, conflicts)
    else:
      worksheet.write(start_row +i, 2, '-')

    if all_statistics[i] is not None and  'propagations' in all_statistics[i].keys():
      propagations = int(all_statistics[i].get_key_value('propagations'))
      worksheet.write(start_row +i, 3, propagations)
    else:
      worksheet.write(start_row +i, 3, '-')

    
    

  workbook.close()


### **Experiment.run**

In [ ]:
time_limit = 300
instance_num =np.linspace(1, 40, 40, dtype=int)
instance_num = [1,2,3,4,5]
num_run = 1

In [ ]:
for i in range(num_run):
    print('--- run num : ', i + 1, '***\n\n')
    solve_instances(instance_num, time_limit, rotation=False, symmetry_breaking=False, verbose=False, count=i + 1)
    solve_instances(instance_num, time_limit, rotation=False, symmetry_breaking=True, verbose=False, count=i + 1)
    solve_instances(instance_num, time_limit, rotation=True, symmetry_breaking=False, verbose=True, count=i + 1)
    solve_instances(instance_num, time_limit, rotation=True, symmetry_breaking=True, verbose=True, count=i + 1)